###Fuente de la data

Los micro-datos gestionados en este notebook son tomados de www.datos.gov.co y corresponden a datos cuya fuente primaria es el Ministerio de Defensa Nacional de Colombia

In [1]:
import pandas as pd

###Archivo nxbk-nikm.csv

Contiene información sobre incautaciones de base de coca.  Incluye el número de kilos de la pasta/base de cocaína incautada por la Fuerza Pública en el ejercicio de sus funciones. Entiéndase por Pasta/base de cocaína lo siguiente: La pasta de cocaína es el primer subproducto que se obtiene en la fase inicial de la extracción de los alcaloides de la hoja de coca, a partir de combustibles y ácido sulfúrico. Una vez refinada, se le conoce como base de cocaína. Esta refinación se hace mediante la utilización de sustancias oxidantes, de preferencia el permanganato de potasio. En este caso se agrupan, puesto que se reportan indistintamente en el sistema de información de seguridad y defensa.

In [2]:
df = pd.read_csv("https://www.datos.gov.co/resource/nxbk-nikm.csv?$limit=500000", dtype={'cod_muni': str})
df.head()

,fecha_hecho,cod_depto,departamento,cod_muni,municipio,cantidad,unidad
0,2024-07-31T00:00:00.000,5.0,ANTIOQUIA,ANTIOQUIA,MEDELLIN,0.030,KILOGRAMO
1,2024-07-31T00:00:00.000,5.0,ANTIOQUIA,ANTIOQUIA,MEDELLIN,0.034,KILOGRAMO
2,2024-07-31T00:00:00.000,5.0,ANTIOQUIA,ANTIOQUIA,MEDELLIN,0.020,KILOGRAMO
3,2024-07-31T00:00:00.000,5.0,ANTIOQUIA,ANTIOQUIA,RIONEGRO,0.004,KILOGRAMO
4,2024-07-31T00:00:00.000,8.0,ATLÁNTICO,ATLÁNTICO,BARRANQUILLA,0.025,KILOGRAMO


##Revisión y limpieza para integrarlo a la base de datos

### - Resumen de la estructura del dataset

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 291830 entries, 0 to 291829
Data columns (total 7 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   fecha_hecho   291830 non-null  object 
 1   cod_depto     291829 non-null  float64
 2   departamento  291825 non-null  object 
 3   cod_muni      291825 non-null  object 
 4   municipio     291825 non-null  object 
 5   cantidad      291830 non-null  float64
 6   unidad        291830 non-null  object 
dtypes: float64(2), object(5)
memory usage: 15.6+ MB


Nota: Existe un error evidente en la columna 'cod_muni', al parecer tiene la misma información que la columna departamentos

In [16]:
if set(df['departamento']) == set(df['cod_muni']):
    print("Las columnas tienen los mismos valores únicos")

Las columnas tienen los mismos valores únicos


### Eliminación de columnas irrelevantes para el proyecto

In [17]:
relevant_cols = ['departamento', 'municipio', 'fecha_hecho', 'cantidad', 'unidad']
cocaine_base_confiscations_MinDefensa = df[relevant_cols]
cocaine_base_confiscations_MinDefensa.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 291830 entries, 0 to 291829
Data columns (total 5 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   departamento  291825 non-null  object 
 1   municipio     291825 non-null  object 
 2   fecha_hecho   291830 non-null  object 
 3   cantidad      291830 non-null  float64
 4   unidad        291830 non-null  object 
dtypes: float64(1), object(4)
memory usage: 11.1+ MB


### Convertir columna fecha_hecho a tipo date

In [18]:
# Hacer una copia explícita del DataFrame
cocaine_base_confiscations_MinDefensa_ = cocaine_base_confiscations_MinDefensa.copy()

# Convertir la columna 'fecha_hecho' a tipo datetime (sin especificar formato)
cocaine_base_confiscations_MinDefensa_['fecha_hecho'] = pd.to_datetime(cocaine_base_confiscations_MinDefensa_['fecha_hecho'], errors='coerce')

# Extraer solo el año
cocaine_base_confiscations_MinDefensa_['fecha_hecho'] = cocaine_base_confiscations_MinDefensa_['fecha_hecho'].dt.year

In [19]:
cocaine_base_confiscations_MinDefensa_['fecha_hecho'].unique()

array([2024, 2023, 2022, 2021, 2020, 2019, 2018, 2017, 2016, 2015, 2014,
       2013, 2012, 2011, 2010], dtype=int32)

### Verificar valores nulos

In [20]:
cocaine_base_confiscations_MinDefensa_.isnull().sum()

,0
departamento,5
municipio,5
fecha_hecho,0
cantidad,0
unidad,0


Nota: Los nulos los abordamos más adelante

### Estandarización de categorizaciones

La estandarización de categorizaciones es el proceso de uniformizar y normalizar los valores de las categorías en un conjunto de datos para asegurar la consistencia y evitar discrepancias. Esto es crucial para la calidad y precisión de los análisis

In [21]:
# Imprimir categorías únicas para columnas de tipo object
categorical_col = ['departamento', 'municipio', 'unidad']
for column in categorical_col:
    print(f"Categorías en la columna '{column}':")
    print(cocaine_base_confiscations_MinDefensa_[column].unique())
    print()


Categorías en la columna 'departamento':
['ANTIOQUIA' 'ATLÁNTICO' 'BOGOTÁ D.C.' 'CALDAS' 'CAUCA' 'CESAR' 'HUILA'
 'LA GUAJIRA' 'MAGDALENA' 'NORTE DE SANTANDER' 'PUTUMAYO' 'QUINDÍO'
 'RISARALDA' 'SANTANDER' 'SUCRE' 'TOLIMA' 'VALLE DEL CAUCA' 'BOLÍVAR' nan
 'GUAVIARE' 'META' 'NARIÑO' 'BOYACÁ' 'CUNDINAMARCA' 'AMAZONAS' 'CASANARE'
 'CAQUETÁ' 'CÓRDOBA' 'CHOCÓ' 'VICHADA' 'ARAUCA' 'GUAINÍA' 'VAUPÉS'
 'SAN ANDRÉS ISLAS' 'QUINDIO' 'CAQUETA' 'ATLANTICO' 'BOLIVAR'
 'BOGOTA D.C.' 'CORDOBA' 'ECUADOR' 'CHOCO' 'BOYACA' 'PERU' 'GUAINIA'
 'CHILE' 'BRASIL' 'AGUAS INTERNACIONALES' 'ESPAÑA' 'SAN ANDRES ISLAS'
 'VAUPES']

Categorías en la columna 'municipio':
['MEDELLIN' 'RIONEGRO' 'BARRANQUILLA' 'BOGOTA, D.C.' 'RIOSUCIO' 'BALBOA'
 'PAILITAS' 'RIVERA' 'FONSECA' 'ALGARROBO' 'VILLA DEL ROSARIO'
 'SAN MIGUEL' 'VALLE DEL GUAMUEZ' 'MONTENEGRO' 'PEREIRA' 'BUCARAMANGA'
 'SAN JUAN DE BETULIA' 'AMBALEMA' 'CARTAGO' 'TRUJILLO' 'BELLO' 'CALDAS'
 'ENVIGADO' 'GIRARDOTA' 'ITAGUI' 'LA ESTRELLA' 'JUAN DE ACOSTA' 'SOLEDAD'


- Borrar espacios en blanco al principio y al final, cambiar a mayúsculas, remover acentos y eliminar signos extraños

In [22]:
import unicodedata

def remove_accents_and_special_chars(input_str):
    # Normalizar la cadena a NFKD
    nfkd_form = unicodedata.normalize('NFKD', input_str)

    # Eliminar acentos
    no_accents = ''.join([c for c in nfkd_form if not unicodedata.combining(c)])

    # Definir caracteres no deseados
    unwanted_chars = [',', ';', '!', '?', '#', '$', '%']

    # Eliminar caracteres no deseados
    cleaned_str = ''.join([c for c in no_accents if c not in unwanted_chars])

    # Remover espacios en blanco al principio y al final, y convertir a mayúsculas
    result = cleaned_str.strip().upper()

    return result

In [23]:
# Aplicar la función a todas las columnas categóricas
cocaine_base_confiscations_MinDefensa_[categorical_col] = cocaine_base_confiscations_MinDefensa_[categorical_col].astype(str)
for col in categorical_col:
    cocaine_base_confiscations_MinDefensa_[col] = cocaine_base_confiscations_MinDefensa_[col].apply(remove_accents_and_special_chars)

- Codificación de algunas variables categóricas

### Crear la columna de "codigo_dane" con los códigos reales de de cada municipio

Los códigos reales de los municipios de Colombia, están almacenados en la base de datos PostgreSQL del proyecto, en la tabla municipalities dentro del campo dept_mpio_code, junto con la informacion necesaria para georeferenciar todos los municipios y departamentos de Colombia. Este campo guarda el código del municipio en un formato string de exactamente 5 caracteres.

Para hacer esta verificación, previamente exportamos desde la base de datos PostgreSQL un DataFrame con los siguientes campos: dept_name, mpio_name y dept_mpio_code, los cuales contienen la información de los departamentos y municipios oficiales, junto con sus respectivos códigos. Este DataFrame se carga en la siguiente celda y se utiliza para comparar con la columna "codigo_mpio"

In [24]:
dept_mpios_codes = pd.read_csv("/content/drive/MyDrive/analytics_data_proyect/deptos_mupios.csv", index_col=0, dtype={'dept_mpio_code': str})
print(dept_mpios_codes.info())
dept_mpios_codes.head()

<class 'pandas.core.frame.DataFrame'>
Index: 1121 entries, 0 to 1120
Data columns (total 3 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   dept_mpio_code  1121 non-null   object
 1   dept_name       1121 non-null   object
 2   mupio_name      1121 non-null   object
dtypes: object(3)
memory usage: 35.0+ KB
None


,dept_mpio_code,dept_name,mupio_name
0,97001,VAUPES,MITU
1,97161,VAUPES,CARURU
2,97511,VAUPES,PACOA
3,97666,VAUPES,TARAIRA
4,97777,VAUPES,PAPUNAHUA


- Comparar que los nombres de los departamentos en cada dataframe esten escritos correctamente

Se aplica la función para racionalizar columnas categoricas (Borrar espacios en blanco al principio y al final, cambiar a mayúsculas, remover acentos y eliminar signos extraños) al df 'dept_mpiios_codes'


In [25]:
# Aplicar función a columnas 'dept_name', 'mupio_name'
for col in dept_mpios_codes[['dept_name', 'mupio_name']]:
    dept_mpios_codes[col] = dept_mpios_codes[col].apply(remove_accents_and_special_chars)

In [26]:
# Función para comparar listas y mostrar diferencias
def compare_lists(df1_col, df2_col, label1, label2):
    # Extraer listas únicas y normalizar
    list1 = set(df1_col.str.strip().str.upper().unique())
    list2 = set(df2_col.str.strip().str.upper().unique())

    # Encontrar diferencias
    only_in_list1 = list1 - list2
    only_in_list2 = list2 - list1

    # Imprimir resultados
    print(f"{label1} que no están en {label2}:")
    print(only_in_list1)
    # print(f"{label2} que no están en {label1}:")
    # print(only_in_list2)
    # print()


  - Comparar nombres de departamentos en los dataframes

In [27]:
# Comparar listas de departamento
compare_lists(cocaine_base_confiscations_MinDefensa_['departamento'], dept_mpios_codes['dept_name'],
              "Departamentos en cocaine_base_confiscations_MinDefensa_", "Departamentos dept_mpios_codes")

Departamentos en cocaine_base_confiscations_MinDefensa_ que no están en Departamentos dept_mpios_codes:
{'ECUADOR', 'PERU', 'AGUAS INTERNACIONALES', 'ESPANA', 'SAN ANDRES ISLAS', 'BRASIL', 'NAN', 'CHILE'}


  Nota: Como departamentos en df figuran incautaciones hechas en el exterior: 'ECUADOR', 'PERU', 'AGUAS INTERNACIONALES', 'ESPANA', 'SAN ANDRES ISLAS', 'BRASIL', 'NAN', 'CHILE', además el departamento SAN ANDRES ISLAS está escrito en forma diferente a su nombre real
  
  - Solucionar discrepancias en nombres de departamento SAN ANDRES ISLAS

In [28]:
# Diccionario de mapeo basado en los resultados de la comparación departamentos
depto_mapping = {'SAN ANDRES ISLAS': 'ARCHIPIELAGO DE SAN ANDRES PROVIDENCIA Y SANTA CATALINA'}

# Reemplazar los nombres incorrectos  el dataframe cocaine_base_confiscations_MinDefensa_
cocaine_base_confiscations_MinDefensa_['departamento'] = cocaine_base_confiscations_MinDefensa_['departamento'].replace(depto_mapping)

 - Eliminar confiscaciones hechas en el exterior porque no interesan para nuestro propósito. Incluye los NAN

In [29]:
# Verificar cuantos registros figuran a nombre de 'VENEZUELA', 'PERU', 'BRASIL', 'ESPANA'
confiscation_overseas = cocaine_base_confiscations_MinDefensa_[cocaine_base_confiscations_MinDefensa_['departamento'].isin(['ECUADOR', 'PERU', 'AGUAS INTERNACIONALES', 'ESPANA', 'SAN ANDRES ISLAS', 'BRASIL', 'NAN', 'CHILE'])]
confiscation_overseas['departamento'].value_counts()

,count
departamento,
PERU,8
NAN,5
CHILE,3
BRASIL,2
ECUADOR,1
AGUAS INTERNACIONALES,1
ESPANA,1


In [30]:
# Se procede a eliminar esos registros
cocaine_base_confiscations_MinDefensa_ = cocaine_base_confiscations_MinDefensa_[~(cocaine_base_confiscations_MinDefensa_['departamento'].isin(['ECUADOR', 'PERU', 'AGUAS INTERNACIONALES', 'ESPANA', 'SAN ANDRES ISLAS', 'BRASIL', 'NAN', 'CHILE']))]

  - Comparar nuevamente las listas de departamentos

In [31]:
# Comparar nuevamente las listas de departamento
compare_lists(cocaine_base_confiscations_MinDefensa_['departamento'], dept_mpios_codes['dept_name'],
              "Departamentos en cocaine_base_confiscations_MinDefensa_", "Departamentos dept_mpios_codes")

Departamentos en cocaine_base_confiscations_MinDefensa_ que no están en Departamentos dept_mpios_codes:
set()


 ### -  Aplicar coincidencia difusa para combinacion departamento + municipio en los dos dataframe

Realizar coincidencia difusa (fuzzy matching), significa que puede comparar dos cadenas de texto (strings) y medir su similitud, incluso si no son exactamente iguales. Esta técnica es muy útil en este caso porque si bien los nombres de los departamentos estan debidamente ajustados en los dos df,  los nombres de los municipios pueden tener diferencias (errores de tipeo, variantes en nombres, etc.).

El utilizar la columna del nombre del departamento, es importante en este caso porque en colombia existen municipios con el mismo nombre.


 - Instalar la thefuzz en el entorno de google colab, para realizar coincidencia difusa

In [32]:
!pip install thefuzz

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 25.3 MB/s eta 0:00:00


- Aplicar thefuzz para coincidencia difusa

In [33]:
# Crear una clave única de departamento + municipio en ambos datasets
cocaine_base_confiscations_MinDefensa_['dept_mpio'] = cocaine_base_confiscations_MinDefensa_['departamento'] + '_' + cocaine_base_confiscations_MinDefensa_['municipio']
dept_mpios_codes['dept_mpio'] = dept_mpios_codes['dept_name'] + '_' + dept_mpios_codes['mupio_name']

In [34]:
# Crear un diccionario de municipios y códigos basado en dept_mpios_codes
municipios_dict = dict(zip(dept_mpios_codes['dept_mpio'], dept_mpios_codes['dept_mpio_code']))

In [35]:
from thefuzz import process

# Funcion para Usar fuzzy matching (thefuzz)
def get_best_match(row, municipios_dict, threshold=80):
    dept_mpio_sexual = row['departamento'] + '_' + row['municipio']

    # Buscar la mejor coincidencia en dept_mpios_codes usando fuzzy matching
    best_match, score = process.extractOne(dept_mpio_sexual, municipios_dict.keys())

    # Si la similitud supera el umbral definido, devolver el código mapeado
    if score >= threshold:
        return municipios_dict[best_match]
    else:
        return 'SIN DATOS'


In [36]:
# Aplicar la función para encontrar la mejor coincidencia
cocaine_base_confiscations_MinDefensa_['codigo_dane'] = cocaine_base_confiscations_MinDefensa_.apply(
    lambda row: get_best_match(row, municipios_dict, threshold=80), axis=1)

  - Verificar resultados obtenidos

In [38]:
cocaine_base_confiscations_MinDefensa_.head()

,departamento,municipio,fecha_hecho,cantidad,unidad,dept_mpio,codigo_dane
0,ANTIOQUIA,MEDELLIN,2024,0.030,KILOGRAMO,ANTIOQUIA_MEDELLIN,05001
1,ANTIOQUIA,MEDELLIN,2024,0.034,KILOGRAMO,ANTIOQUIA_MEDELLIN,05001
2,ANTIOQUIA,MEDELLIN,2024,0.020,KILOGRAMO,ANTIOQUIA_MEDELLIN,05001
3,ANTIOQUIA,RIONEGRO,2024,0.004,KILOGRAMO,ANTIOQUIA_RIONEGRO,05615
4,ATLANTICO,BARRANQUILLA,2024,0.025,KILOGRAMO,ATLANTICO_BARRANQUILLA,08001


- Verificar que los códigos de municipios que quedaron en el dataset correspondan solamente a códigos reales

In [39]:
# Comparar listas de códigos
compare_lists(cocaine_base_confiscations_MinDefensa_['codigo_dane'], dept_mpios_codes['dept_mpio_code'],
              "Códigos de municipios en cocaine_base_confiscations_MinDefensa_", "Códigos de municipios dept_mpios_codes")

Códigos de municipios en cocaine_base_confiscations_MinDefensa_ que no están en Códigos de municipios dept_mpios_codes:
{'SIN DATOS'}


In [40]:
# Verificar cuantos municipios quedaron sin código
len(cocaine_base_confiscations_MinDefensa_[cocaine_base_confiscations_MinDefensa_['codigo_dane']== 'SIN DATOS'])

1

In [41]:
# Verificar cuales municipios quedaron sin código
cocaine_base_confiscations_MinDefensa_[cocaine_base_confiscations_MinDefensa_['codigo_dane']== 'SIN DATOS']

,departamento,municipio,fecha_hecho,cantidad,unidad,dept_mpio,codigo_dane
151867,GUAINIA,MAPIRIPANA,2015,0.15,KILOGRAMO,GUAINIA_MAPIRIPANA,SIN DATOS


In [42]:
# Listado de municipios de Guania
dept_mpios_codes[dept_mpios_codes['dept_name'] == 'GUAINIA']

,dept_mpio_code,dept_name,mupio_name,dept_mpio
1092,94001,GUAINIA,INIRIDA,GUAINIA_INIRIDA
1093,94343,GUAINIA,BARRANCOMINAS,GUAINIA_BARRANCOMINAS
1094,94883,GUAINIA,SAN FELIPE,GUAINIA_SAN FELIPE
1095,94884,GUAINIA,PUERTO COLOMBIA,GUAINIA_PUERTO COLOMBIA
1096,94885,GUAINIA,LA GUADALUPE,GUAINIA_LA GUADALUPE
1097,94886,GUAINIA,CACAHUAL,GUAINIA_CACAHUAL
1098,94887,GUAINIA,PANA PANA,GUAINIA_PANA PANA
1099,94888,GUAINIA,MORICHAL,GUAINIA_MORICHAL


In [43]:
# Como no hay ningun en Guania de nombre MAPIRIAPANA eliminamos el registro
cocaine_base_confiscations_MinDefensa_ = cocaine_base_confiscations_MinDefensa_[~(cocaine_base_confiscations_MinDefensa_['codigo_dane']== 'SIN DATOS')]

 - Verificar que los códigos de municipios que quedaron en el dataset correspondan solamente a códigos reales

In [44]:
# Comparar listas de códigos
compare_lists(cocaine_base_confiscations_MinDefensa_['codigo_dane'], dept_mpios_codes['dept_mpio_code'],
              "Códigos de municipios en cocaine_base_confiscations_MinDefensa_", "Códigos de municipios dept_mpios_codes")

Códigos de municipios en cocaine_base_confiscations_MinDefensa_ que no están en Códigos de municipios dept_mpios_codes:
set()


### Procesamiento final como preparación para integrarlo a la bd de datos del proyecto

In [52]:
# Eliminar columnas innecesarias
columns_to_drop = ['departamento', 'municipio', 'dept_mpio']
final_cocaine_base_confiscations_MinDefensa = cocaine_base_confiscations_MinDefensa_.drop(columns=columns_to_drop)

In [51]:
cocaine_base_confiscations_MinDefensa_.columns

Index(['departamento', 'municipio', 'fecha_hecho', 'cantidad', 'unidad',
       'dept_mpio', 'codigo_dane'],
      dtype='object')

In [54]:
# Adicionar columna para trazabilidad de la fuente
final_cocaine_base_confiscations_MinDefensa['source_id'] = 30

In [55]:
final_cocaine_base_confiscations_MinDefensa.columns

Index(['fecha_hecho', 'cantidad', 'unidad', 'codigo_dane', 'source_id'], dtype='object')

In [56]:
# Ajustar nombre de columnas

# Definir el diccionario de traducción
translation_map = {
    'codido_dane': 'dane_code',
    'fecha_hecho': 'year_of_incident',
    'cantidad': 'amount',
    'unidad': 'unit',
    'source_id': 'source_id'
}

# Renombrar las columnas
final_cocaine_base_confiscations_MinDefensa.rename(columns=translation_map, inplace=True)

In [57]:
#Estructura final del dataset a integrar a la base de datos
final_cocaine_base_confiscations_MinDefensa.info()

<class 'pandas.core.frame.DataFrame'>
Index: 291808 entries, 0 to 291829
Data columns (total 5 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   year_of_incident  291808 non-null  int32  
 1   amount            291808 non-null  float64
 2   unit              291808 non-null  object 
 3   codigo_dane       291808 non-null  object 
 4   source_id         291808 non-null  int64  
dtypes: float64(1), int32(1), int64(1), object(2)
memory usage: 12.2+ MB


## Salvar en archivo csv en el drive

In [58]:
# Guardar en archivos CSV en el drive
final_cocaine_base_confiscations_MinDefensa.to_csv('/content/drive/MyDrive/analytics_data_proyect/initial_transformation/cocaine_base_confiscations_MinDefensa.csv', index=False)